In [ ]:
# Import required libraries and dependencies

In [130]:
#import keras as krs
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import tensorflow as tf
import sklearn as sk
import pandas as pd
import numpy as np
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(precision=4, suppress=True)
import os
import json
import requests
from dotenv import load_dotenv
%matplotlib inline
from pathlib import Path

In [131]:
def assess_NA(data):
    """
    Returns a pandas dataframe denoting the total number of NA values and the percentage of NA values in each column.
    The column names are noted on the index.
    
    Parameters
    ----------
    data: dataframe
    """
    # pandas series denoting features and the sum of their null values
    null_sum = data.isnull().sum()# instantiate columns for missing data
    total = null_sum.sort_values(ascending=False)
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False)
    
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])
    
    # drop rows that don't have any missing data; omit if you want to keep all rows
    df_NA = df_NA[ (df_NA.T != 0).any() ]
    
    return df_NA

# us_balance_annual

In [142]:
us_balance_annual = pd.read_csv(
    Path('us-balance-annual.csv'),
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

In [143]:
us_balance_annual.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity
0,A,45846,USD,2008,Q4,2008-10-31,2008-12-19,2009-12-21,363000000.0,371000000.0,...,NaN,1.330000e+09,2.125000e+09,3.118000e+09,4448000000,7.416000e+09,-7.470000e+09,2.791000e+09,2.559000e+09,7007000000
1,A,45846,USD,2009,Q4,2009-10-31,2009-12-21,2010-12-20,346000000.0,346000000.0,...,1.000000e+06,1.123000e+09,2.904000e+09,3.975000e+09,5098000000,7.558000e+09,-7.627000e+09,2.760000e+09,2.514000e+09,7612000000
2,A,45846,USD,2010,Q4,2010-10-31,2010-12-20,2011-12-16,347000000.0,353000000.0,...,1.501000e+09,3.083000e+09,2.190000e+09,3.377000e+09,6460000000,7.910000e+09,-8.038000e+09,3.444000e+09,3.236000e+09,9696000000
3,A,45846,USD,2011,Q4,2011-10-31,2011-12-16,2012-12-20,347000000.0,355000000.0,...,2.530000e+08,1.837000e+09,1.932000e+09,2.904000e+09,4741000000,8.271000e+09,-8.535000e+09,4.456000e+09,4.316000e+09,9057000000
4,A,45846,USD,2012,Q4,2012-10-31,2012-12-20,2013-12-19,348000000.0,353000000.0,...,2.500000e+08,1.893000e+09,2.112000e+09,3.458000e+09,5351000000,8.495000e+09,-8.707000e+09,5.505000e+09,5.185000e+09,10536000000


In [144]:
supa_balance = us_balance_annual.drop(['SimFinId', 'Currency'], axis=1)

In [145]:
balance_data = supa_balance.groupby(['Ticker','Fiscal Year']).mean()


In [146]:
balance_data

Shares (Basic)  Shares (Diluted)  \
Ticker Fiscal Year                                     
A      2008            363000000.0       371000000.0   
       2009            346000000.0       346000000.0   
       2010            347000000.0       353000000.0   
       2011            347000000.0       355000000.0   
       2012            348000000.0       353000000.0   
...                            ...               ...   
ZYXI   2015             31271234.0        31271234.0   
       2016             31271234.0        31271234.0   
       2017             32156000.0        33196000.0   
       2018             32503000.0        34043000.0   
       2019             32439000.0        33963000.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Fiscal Year                                                    
A      2008                                            1.429000e+09   
       2009                                            2.493000e+09   
       2010                                            2.649000e+09   
       2011                                            3.527000e+09   
       2012                                            2.351000e+09   
...                                                             ...   
ZYXI   2015                                            8.000000e+03   
       2016                                            2.470000e+05   
       2017                                            5.565000e+06   
       2018                                            1.012800e+07   
       2019                                            1.404000e+07   

                    Accounts & Notes Receivable   Inventories  \
Ticker Fiscal Year                                              
A      2008                         770000000.0  6.460000e+08   
       2009                         595000000.0  5.520000e+08   
       2010                         869000000.0  7.160000e+08   
       2011                         860000000.0  8.980000e+08   
       2012                         923000000.0  1.014000e+09   
...                                         ...           ...   
ZYXI   2015                           2426000.0  3.050000e+05   
       2016                           3028000.0  1.070000e+05   
       2017                           2185000.0  4.230000e+05   
       2018                           2791000.0  8.370000e+05   
       2019                           5833000.0  2.378000e+06   

                    Total Current Assets  Property, Plant & Equipment, Net  \
Ticker Fiscal Year                                                           
A      2008                 3.182000e+09                      8.240000e+08   
       2009                 3.961000e+09                      8.450000e+08   
       2010                 6.169000e+09                      9.800000e+08   
       2011                 5.569000e+09                      1.006000e+09   
       2012                 4.629000e+09                      1.164000e+09   
...                                  ...                               ...   
ZYXI   2015                 2.766000e+06                      8.010000e+05   
       2016                 3.422000e+06                      5.800000e+05   
       2017                 8.371000e+06                      1.880000e+05   
       2018                 1.432400e+07                      8.190000e+05   
       2019                 2.256600e+07                      8.580000e+05   

                    Long Term Investments & Receivables  \
Ticker Fiscal Year                                        
A      2008                                 206000000.0   
       2009                                 163000000.0   
       2010                                 142000000.0   
       2011                                 117000000.0   
       2012                                 109000000.0   
...                                                 ...   
ZYXI   2015                       

In [147]:
balance_data.isnull().sum()

Shares (Basic)                                       203
Shares (Diluted)                                     203
Cash, Cash Equivalents & Short Term Investments       65
Accounts & Notes Receivable                         1564
Inventories                                         6048
Total Current Assets                                   2
Property, Plant & Equipment, Net                     364
Long Term Investments & Receivables                13770
Other Long Term Assets                               250
Total Noncurrent Assets                               82
Total Assets                                           0
Payables & Accruals                                   74
Short Term Debt                                     6490
Total Current Liabilities                              1
Long Term Debt                                      4102
Total Noncurrent Liabilities                         217
Total Liabilities                                      0
Share Capital & Additional Paid

In [148]:
assess_NA(balance_data)

,Number of NA,Percent NA
Long Term Investments & Receivables,13770,71.51
Treasury Stock,10128,52.60
Short Term Debt,6490,33.71
Inventories,6048,31.41
Long Term Debt,4102,21.30
Accounts & Notes Receivable,1564,8.12
Retained Earnings,730,3.79
"Property, Plant & Equipment, Net",364,1.89
Other Long Term Assets,250,1.30
Total Noncurrent Liabilities,217,1.13


In [149]:
balance_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 19255 entries, ('A', 2008) to ('ZYXI', 2019)
Data columns (total 22 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Shares (Basic)                                   19052 non-null  float64
 1   Shares (Diluted)                                 19052 non-null  float64
 2   Cash, Cash Equivalents & Short Term Investments  19190 non-null  float64
 3   Accounts & Notes Receivable                      17691 non-null  float64
 4   Inventories                                      13207 non-null  float64
 5   Total Current Assets                             19253 non-null  float64
 6   Property, Plant & Equipment, Net                 18891 non-null  float64
 7   Long Term Investments & Receivables              5485 non-null   float64
 8   Other Long Term Assets                           19005 non-null  float64
 9   Total Non

# us_cashflow_annual_df

In [151]:

us_cashflow_annual_df = pd.read_csv(
    Path('us-cashflow-annual.csv'),
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)


In [152]:
us_cashflow_annual_df.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
0,A,45846,USD,2008,FY,2008-10-31,2008-12-19,2010-12-20,363000000.0,371000000.0,...,756000000,-140000000.0,150000000.0,-1.720000e+08,-3.850000e+08,NaN,1.600000e+07,-790000000.0,-7.880000e+08,-421000000
1,A,45846,USD,2009,FY,2009-10-31,2009-12-21,2011-12-16,346000000.0,346000000.0,...,408000000,-127000000.0,94000000.0,4.300000e+07,-4.000000e+06,NaN,7.480000e+08,-86000000.0,6.470000e+08,1074000000
2,A,45846,USD,2010,FY,2010-10-31,2010-12-20,2012-12-20,347000000.0,353000000.0,...,718000000,-114000000.0,38000000.0,-1.108000e+09,-1.174000e+09,0.0,7.180000e+08,-112000000.0,6.010000e+08,170000000
3,A,45846,USD,2011,FY,2011-10-31,2011-12-16,2013-12-19,347000000.0,355000000.0,...,1260000000,-170000000.0,16000000.0,-9.700000e+07,1.294000e+09,0.0,-1.500000e+09,-193000000.0,-1.693000e+09,878000000
4,A,45846,USD,2012,FY,2012-10-31,2012-12-20,2014-12-22,348000000.0,353000000.0,...,1228000000,-194000000.0,5000000.0,-2.257000e+09,-2.366000e+09,-104000000.0,1.500000e+08,-72000000.0,-3.500000e+07,-1174000000


In [153]:
dirty_cash_data = us_cashflow_annual_df.drop(['SimFinId', 'Currency'], axis=1)
cash_flow = dirty_cash_data.groupby(['Ticker','Fiscal Year']).mean()
cash_flow

Shares (Basic)  Shares (Diluted)  \
Ticker Fiscal Year                                     
A      2008            363000000.0       371000000.0   
       2009            346000000.0       346000000.0   
       2010            347000000.0       353000000.0   
       2011            347000000.0       355000000.0   
       2012            348000000.0       353000000.0   
...                            ...               ...   
ZYXI   2015             31271234.0        31271234.0   
       2016             31271234.0        31271234.0   
       2017             32156000.0        33196000.0   
       2018             32503000.0        34043000.0   
       2019             32439000.0        33963000.0   

                    Net Income/Starting Line  Depreciation & Amortization  \
Ticker Fiscal Year                                                          
A      2008                     6.930000e+08                  201000000.0   
       2009                    -3.100000e+07                  162000000.0   
       2010                     6.840000e+08                  202000000.0   
       2011                     1.012000e+09                  253000000.0   
       2012                     1.153000e+09                  301000000.0   
...                                      ...                          ...   
ZYXI   2015                    -2.934000e+06                     424000.0   
       2016                     6.900000e+04                     435000.0   
       2017                     7.365000e+06                     286000.0   
       2018                     9.552000e+06                     448000.0   
       2019                     9.492000e+06                     778000.0   

                    Non-Cash Items  Change in Working Capital  \
Ticker Fiscal Year                                              
A      2008             41000000.0               -179000000.0   
       2009            215000000.0                 62000000.0   
       2010           -116000000.0                -52000000.0   
       2011            219000000.0               -224000000.0   
       2012            -50000000.0               -176000000.0   
...                            ...                        ...   
ZYXI   2015              1198000.0                  1653000.0   
       2016               192000.0                  1073000.0   
       2017              1771000.0                 -1162000.0   
       2018                15000.0                  -607000.0   
       2019               152000.0                 -4119000.0   

                    Change in Accounts Receivable  Change in Inventories  \
Ticker Fiscal Year                                                         
A      2008                           -44000000.0            -14000000.0   
       2009                           193000000.0             47000000.0   
       2010                          -166000000.0            -51000000.0   
       2011                            11000000.0           -208000000.0   
       2012                            19000000.0            -52000000.0   
...                                           ...                    ...   
ZYXI   2015                                   NaN                    NaN   
       2016                                   NaN                    NaN   
       2017                                   NaN                    NaN   
       2018                                   NaN                    NaN   
       2019                                   NaN                    NaN   

                    Change in Accounts Payable  Change in Other  \
Ticker Fiscal Year                                                
A      2008                        -21000000.0     -100000000.0   
       2009                         -7000000.0     -171000000.0   
       2010                        113000000.0       52000000.0   
       2011                        -35000000.0        8000000.0   
       2012                        -31000000.0     -

In [154]:
cash_flow.isnull().sum()

Shares (Basic)                                 203
Shares (Diluted)                               203
Net Income/Starting Line                        69
Depreciation & Amortization                    415
Non-Cash Items                                  82
Change in Working Capital                       97
Change in Accounts Receivable                17443
Change in Inventories                        17876
Change in Accounts Payable                   17818
Change in Other                              17269
Net Cash from Operating Activities               0
Change in Fixed Assets & Intangibles           411
Net Change in Long Term Investment           11994
Net Cash from Acquisitions & Divestitures     7787
Net Cash from Investing Activities              78
Dividends Paid                                9529
Cash from (Repayment of) Debt                 2400
Cash from (Repurchase of) Equity              2281
Net Cash from Financing Activities              63
Net Change in Cash             

In [155]:
assess_NA(cash_flow)

,Number of NA,Percent NA
Change in Inventories,17876,92.92
Change in Accounts Payable,17818,92.62
Change in Accounts Receivable,17443,90.67
Change in Other,17269,89.77
Net Change in Long Term Investment,11994,62.35
Dividends Paid,9529,49.53
Net Cash from Acquisitions & Divestitures,7787,40.48
Cash from (Repayment of) Debt,2400,12.48
Cash from (Repurchase of) Equity,2281,11.86
Depreciation & Amortization,415,2.16


In [156]:
cash_flow.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 19238 entries, ('A', 2008) to ('ZYXI', 2019)
Data columns (total 20 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Shares (Basic)                             19035 non-null  float64
 1   Shares (Diluted)                           19035 non-null  float64
 2   Net Income/Starting Line                   19169 non-null  float64
 3   Depreciation & Amortization                18823 non-null  float64
 4   Non-Cash Items                             19156 non-null  float64
 5   Change in Working Capital                  19141 non-null  float64
 6   Change in Accounts Receivable              1795 non-null   float64
 7   Change in Inventories                      1362 non-null   float64
 8   Change in Accounts Payable                 1420 non-null   float64
 9   Change in Other                            1969 non-null   float64
 10  Net

In [157]:
cash_flow.stack().head(30)

Ticker  Fiscal Year                                           
A       2008         Shares (Basic)                               363000000.0
                     Shares (Diluted)                             371000000.0
                     Net Income/Starting Line                     693000000.0
                     Depreciation & Amortization                  201000000.0
                     Non-Cash Items                                41000000.0
                     Change in Working Capital                   -179000000.0
                     Change in Accounts Receivable                -44000000.0
                     Change in Inventories                        -14000000.0
                     Change in Accounts Payable                   -21000000.0
                     Change in Other                             -100000000.0
                     Net Cash from Operating Activities           756000000.0
                     Change in Fixed Assets & Intangibles        -140000000.0
 

In [158]:
cash_flow.describe()

,Shares (Basic),Shares (Diluted),Net Income/Starting Line,Depreciation & Amortization,Non-Cash Items,Change in Working Capital,Change in Accounts Receivable,Change in Inventories,Change in Accounts Payable,Change in Other,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
count,1.903500e+04,1.903500e+04,1.916900e+04,1.882300e+04,1.915600e+04,1.914100e+04,1.795000e+03,1.362000e+03,1.420000e+03,1.969000e+03,1.923800e+04,1.882700e+04,7.244000e+03,1.145100e+04,1.916000e+04,9.709000e+03,1.683800e+04,1.695700e+04,1.917500e+04,1.923800e+04
mean,2.452987e+08,2.498697e+08,4.996212e+08,3.720423e+08,1.292793e+08,-5.148521e+07,-1.177328e+08,-2.028515e+08,9.672251e+07,-1.393352e+08,9.469144e+08,-4.221342e+08,-7.033687e+07,-3.546539e+08,-6.345839e+08,-4.162507e+08,1.461146e+08,-2.234022e+08,-2.761579e+08,3.840615e+07
std,9.919641e+08,1.011586e+09,2.320646e+09,1.334056e+09,1.445911e+09,1.060254e+09,6.847983e+08,1.018381e+09,7.183441e+08,2.509454e+09,3.373684e+09,1.686618e+09,2.408776e+09,2.480280e+09,2.770914e+09,1.213826e+09,2.166131e+09,1.500401e+09,2.582945e+09,1.246523e+09
min,2.000000e+00,2.000000e+00,-2.071700e+10,-1.650000e+08,-6.422600e+10,-4.340700e+10,-8.214000e+09,-1.421500e+10,-7.471000e+09,-4.353700e+10,-1.591400e+10,-3.684200e+10,-4.441700e+10,-6.292400e+10,-8.426700e+10,-3.611200e+10,-6.770300e+10,-7.206900e+10,-9.097600e+10,-4.553200e+10
25%,2.620500e+07,2.651560e+07,-2.056000e+06,9.744000e+06,9.846475e+05,-3.663100e+07,-1.461605e+08,-1.319078e+08,-1.800000e+07,-9.600000e+07,1.373050e+07,-2.066620e+08,-4.305350e+07,-1.550000e+08,-3.941268e+08,-2.961480e+08,-2.600000e+07,-8.214300e+07,-1.881875e+08,-2.108250e+07
50%,6.246900e+07,6.338256e+07,5.545800e+07,5.944000e+07,1.618226e+07,-1.729000e+06,-3.180000e+07,-2.168650e+07,1.213800e+07,2.590000e+06,1.502630e+08,-4.513000e+07,-1.355000e+05,-2.040200e+07,-7.970924e+07,-7.564600e+07,0.000000e+00,0.000000e+00,-9.869000e+06,1.394227e+06
75%,1.766700e+08,1.790000e+08,3.111000e+08,2.301770e+08,8.000000e+07,1.292600e+07,8.100000e+06,7.738750e+06,1.047408e+08,1.110000e+08,6.124480e+08,-6.506000e+06,1.215850e+07,0.000000e+00,-7.305169e+06,-1.664000e+07,1.250000e+08,6.546000e+06,2.274050e+07,4.884650e+07
max,2.617490e+10,2.646993e+10,8.179200e+10,3.876200e+10,5.492500e+10,3.425000e+10,8.641000e+09,3.755000e+09,9.943000e+09,1.772400e+10,8.126600e+10,9.269000e+09,7.606500e+10,6.987100e+10,5.948700e+10,1.440000e+05,4.086100e+10,1.910000e+10,3.969600e+10,5.043500e+10


# income-annual

In [160]:
us_income_annual_df = pd.read_csv(
    Path('us-income-annual.csv'),
    
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

In [161]:
us_income_annual_df.tail()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
19250,ZYXI,171401,USD,2015,FY,2015-12-31,2016-03-31,2017-04-17,31271234.0,31271234.0,...,-510000.0,-503000.0,-2991000,NaN,-2991000,57000.0,-2934000,NaN,-2911000,-2911000
19251,ZYXI,171401,USD,2016,FY,2016-12-31,2017-04-18,2018-02-28,31271234.0,31271234.0,...,-556000.0,-352000.0,84000,NaN,84000,-15000.0,69000,NaN,69000,69000
19252,ZYXI,171401,USD,2017,FY,2017-12-31,2018-02-28,2018-02-28,32156000.0,33196000.0,...,-1450000.0,-1450000.0,7494000,NaN,7494000,-129000.0,7365000,NaN,7365000,7365000
19253,ZYXI,171401,USD,2018,FY,2018-12-31,2019-02-26,2019-02-26,32503000.0,34043000.0,...,-154000.0,-154000.0,10216000,NaN,10216000,-664000.0,9552000,NaN,9552000,9552000
19254,ZYXI,171401,USD,2019,FY,2019-12-31,2020-02-27,2020-02-27,32439000.0,33963000.0,...,-5000.0,-5000.0,11061000,880000.0,11941000,-2449000.0,9492000,NaN,9492000,9492000


In [162]:
dirty_income = us_income_annual_df.drop(['SimFinId'], axis=1)
income_annual = dirty_income.groupby(['Ticker','Fiscal Year']).mean()
income_annual

Shares (Basic)  Shares (Diluted)       Revenue  \
Ticker Fiscal Year                                                   
A      2008            363000000.0       371000000.0  5.774000e+09   
       2009            346000000.0       346000000.0  4.481000e+09   
       2010            347000000.0       353000000.0  5.444000e+09   
       2011            347000000.0       355000000.0  6.615000e+09   
       2012            348000000.0       353000000.0  6.858000e+09   
...                            ...               ...           ...   
ZYXI   2015             31271234.0        31271234.0  1.164100e+07   
       2016             31271234.0        31271234.0  1.331300e+07   
       2017             32156000.0        33196000.0  2.343200e+07   
       2018             32503000.0        34043000.0  3.191700e+07   
       2019             32439000.0        33963000.0  4.547200e+07   

                    Cost of Revenue  Gross Profit  Operating Expenses  \
Ticker Fiscal Year                                                      
A      2008           -2.578000e+09  3.196000e+09       -2.401000e+09   
       2009           -2.189000e+09  2.292000e+09       -2.245000e+09   
       2010           -2.514000e+09  2.930000e+09       -2.364000e+09   
       2011           -3.086000e+09  3.529000e+09       -2.458000e+09   
       2012           -3.254000e+09  3.604000e+09       -2.485000e+09   
...                             ...           ...                 ...   
ZYXI   2015           -4.937000e+06  6.704000e+06       -9.185000e+06   
       2016           -3.517000e+06  9.796000e+06       -9.156000e+06   
       2017           -4.819000e+06  1.861300e+07       -9.669000e+06   
       2018           -6.038000e+06  2.587900e+07       -1.550900e+07   
       2019           -8.814000e+06  3.665800e+07       -2.559200e+07   

                    Selling, General & Administrative  Research & Development  \
Ticker Fiscal Year                                                              
A      2008                             -1.697000e+09            -704000000.0   
       2009                             -1.603000e+09            -642000000.0   
       2010                             -1.752000e+09            -612000000.0   
       2011                             -1.809000e+09            -649000000.0   
       2012                             -1.817000e+09            -668000000.0   
...                                               ...                     ...   
ZYXI   2015                             -9.185000e+06                     NaN   
       2016                             -9.156000e+06                     NaN   
       2017                             -9.669000e+06                     NaN   
       2018                             -1.550900e+07                     NaN   
       2019                             -2.559200e+07                     NaN   

                    Depreciation & Amortization  Operating Income (Loss)  \
Ticker Fiscal Year                                                         
A      2008                                 NaN                795000000   
       2009                                 NaN                 47000000   
       2010                                 NaN                566000000   
       2011                                 NaN               1071000000   
       2012                                 NaN               1119000000   
...                                         ...                      ...   
ZYXI   2015                                 NaN                 -2481000   
       2016                                 NaN                   640000   
       2017                                 NaN                  8944000   
       2018                                 NaN                 10370000   
       2019                                 NaN                 11066000   

                    Non-Operating Income (Loss)  Interest Expense, Net  \
Ticker Fiscal Year                        

In [163]:
income_annual.isnull().sum()

Shares (Basic)                                203
Shares (Diluted)                              203
Revenue                                       281
Cost of Revenue                              1838
Gross Profit                                  265
Operating Expenses                             65
Selling, General & Administrative             950
Research & Development                      11650
Depreciation & Amortization                 10699
Operating Income (Loss)                         0
Non-Operating Income (Loss)                   191
Interest Expense, Net                        2156
Pretax Income (Loss), Adj.                      0
Abnormal Gains (Losses)                      7115
Pretax Income (Loss)                            0
Income Tax (Expense) Benefit, Net            1136
Income (Loss) from Continuing Operations        0
Net Extraordinary Gains (Losses)            15139
Net Income                                      0
Net Income (Common)                             0


In [164]:
income_annual.describe()

,Shares (Basic),Shares (Diluted),Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Research & Development,Depreciation & Amortization,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
count,1.905200e+04,1.905200e+04,1.897400e+04,1.741700e+04,1.899000e+04,1.919000e+04,1.830500e+04,7.605000e+03,8.556000e+03,1.925500e+04,1.906400e+04,1.709900e+04,1.925500e+04,1.214000e+04,1.925500e+04,1.811900e+04,1.925500e+04,4.116000e+03,1.925500e+04,1.925500e+04
mean,2.451149e+08,2.496820e+08,6.794764e+09,-4.753004e+09,2.430386e+09,-1.551795e+09,-1.030333e+09,-3.772612e+08,-4.356324e+08,8.509667e+08,-1.147292e+08,-1.266842e+08,7.373757e+08,-1.181633e+08,6.628754e+08,-1.813431e+08,4.939769e+08,4.353438e+07,4.880439e+08,4.851444e+08
std,9.915405e+08,1.011154e+09,2.322183e+10,1.765833e+10,7.667796e+09,4.794611e+09,3.646086e+09,1.278280e+09,1.568861e+09,3.602512e+09,1.275564e+09,4.540450e+08,3.043115e+09,9.193843e+08,3.049554e+09,1.043311e+09,2.297784e+09,6.000995e+08,2.288431e+09,2.287385e+09
min,2.000000e+00,2.000000e+00,-6.741000e+06,-3.946050e+11,-1.280800e+10,-1.087910e+11,-1.087910e+11,-2.601800e+10,-2.921800e+10,-1.347300e+10,-5.912000e+10,-2.575800e+10,-1.489500e+10,-3.426900e+10,-2.869500e+10,-3.653000e+10,-2.071700e+10,-9.559000e+09,-2.235400e+10,-2.280100e+10
25%,2.623300e+07,2.653456e+07,2.733210e+08,-2.892844e+09,1.153652e+08,-1.105516e+09,-7.089900e+08,-1.739250e+08,-2.810698e+08,5.266726e+06,-8.080000e+07,-1.044245e+08,4.955825e+05,-4.085950e+07,-1.680785e+06,-1.109225e+08,-2.005513e+06,-3.838500e+06,-2.121934e+06,-2.614548e+06
50%,6.241700e+07,6.330376e+07,1.329680e+09,-7.544910e+08,5.264455e+08,-3.418300e+08,-1.980000e+08,-5.048900e+07,-7.245050e+07,1.220850e+08,-1.070100e+07,-2.129400e+07,8.890000e+07,-4.184500e+06,7.491900e+07,-1.920000e+07,5.534200e+07,0.000000e+00,5.382300e+07,5.300600e+07
75%,1.765087e+08,1.789880e+08,4.630200e+09,-1.454990e+08,1.733500e+09,-9.037350e+07,-4.665300e+07,-1.388100e+07,-1.516850e+07,5.515500e+08,-3.900000e+04,-8.995000e+05,4.397405e+08,0.000000e+00,4.059500e+08,-2.790000e+05,3.059880e+08,1.341700e+07,3.057000e+08,3.032415e+08
max,2.617490e+10,2.646993e+10,5.239640e+11,1.219000e+09,1.722200e+11,1.110132e+09,0.000000e+00,6.480000e+05,4.962000e+09,1.425170e+11,4.304900e+10,1.537000e+09,1.026960e+11,1.289400e+10,1.026960e+11,3.483100e+10,8.179200e+10,1.591450e+10,8.141700e+10,8.141700e+10


In [165]:
income_annual.stack().head(50)

Ticker  Fiscal Year                                          
A       2008         Shares (Basic)                              3.630000e+08
                     Shares (Diluted)                            3.710000e+08
                     Revenue                                     5.774000e+09
                     Cost of Revenue                            -2.578000e+09
                     Gross Profit                                3.196000e+09
                     Operating Expenses                         -2.401000e+09
                     Selling, General & Administrative          -1.697000e+09
                     Research & Development                     -7.040000e+08
                     Operating Income (Loss)                     7.950000e+08
                     Non-Operating Income (Loss)                 2.000000e+07
                     Interest Expense, Net                      -1.000000e+07
                     Pretax Income (Loss), Adj.                  8.150000e+08
  

In [166]:
assess_NA(income_annual)

,Number of NA,Percent NA
Net Extraordinary Gains (Losses),15139,78.62
Research & Development,11650,60.50
Depreciation & Amortization,10699,55.56
Abnormal Gains (Losses),7115,36.95
"Interest Expense, Net",2156,11.20
Cost of Revenue,1838,9.55
"Income Tax (Expense) Benefit, Net",1136,5.90
"Selling, General & Administrative",950,4.93
Revenue,281,1.46
Gross Profit,265,1.38


In [167]:
income_annual.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 19255 entries, ('A', 2008) to ('ZYXI', 2019)
Data columns (total 20 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Shares (Basic)                            19052 non-null  float64
 1   Shares (Diluted)                          19052 non-null  float64
 2   Revenue                                   18974 non-null  float64
 3   Cost of Revenue                           17417 non-null  float64
 4   Gross Profit                              18990 non-null  float64
 5   Operating Expenses                        19190 non-null  float64
 6   Selling, General & Administrative         18305 non-null  float64
 7   Research & Development                    7605 non-null   float64
 8   Depreciation & Amortization               8556 non-null   float64
 9   Operating Income (Loss)                   19255 non-null  int64  
 10  Non-Operating I

# shareprices

In [169]:
us_shareprice_df = pd.read_csv(
    Path('us-shareprices-daily.csv'),
    
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)


us_shareprice_df

,Ticker,SimFinId,Date,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
0,GOOG,18,2014-03-27,568.00,552.92,568.00,558.46,558.46,NaN,13100,336050831.0
1,GOOG,18,2014-03-28,561.20,558.67,566.43,559.99,559.99,NaN,41100,336050831.0
2,GOOG,18,2014-03-31,566.89,556.93,567.00,556.97,556.97,NaN,10800,674462000.0
3,GOOG,18,2014-04-01,558.71,558.71,568.45,567.16,567.16,NaN,7900,674462000.0
4,GOOG,18,2014-04-02,565.11,562.19,604.83,567.00,567.00,NaN,146700,674462000.0
...,...,...,...,...,...,...,...,...,...,...,...
5982671,GOEV,1104613,2020-02-11,10.27,10.27,10.27,10.27,10.27,NaN,2615,8701731.0
5982672,GOEV,1104613,2020-02-12,10.27,10.27,10.27,10.27,10.27,NaN,2,8701731.0
5982673,GOEV,1104613,2020-02-13,10.27,10.27,10.27,10.27,10.27,NaN,326,8701731.0
5982674,GOEV,1104613,2020-02-14,10.38,10.24,10.38,10.28,10.28,NaN,490298,8701731.0


In [170]:
ticker_data = us_shareprice_df.groupby(['Ticker', 'Date']).mean()
ticker_data

SimFinId   Open    Low   High  Close  Adj. Close  Dividend  \
Ticker Date                                                                     
A      2007-01-03     45846  34.99  34.05  35.48  34.30       22.72       NaN   
       2007-01-04     45846  34.30  33.46  34.60  34.41       22.80       NaN   
       2007-01-05     45846  34.30  34.00  34.40  34.09       22.58       NaN   
       2007-01-08     45846  33.98  33.68  34.08  33.97       22.51       NaN   
       2007-01-09     45846  34.08  33.63  34.32  34.01       22.53       NaN   
...                     ...    ...    ...    ...    ...         ...       ...   
ZYXI   2020-02-11    171401  10.02   9.55  10.02   9.88        9.88       NaN   
       2020-02-12    171401   9.92   9.92  10.50  10.32       10.32       NaN   
       2020-02-13    171401  10.24  10.10  10.36  10.26       10.26       NaN   
       2020-02-14    171401  10.20   9.95  10.33  10.16       10.16       NaN   
       2020-02-18    171401  10.15  10.01  10.34  10.18       10.18       NaN   

                    Volume  Shares Outstanding  
Ticker Date                                     
A      2007-01-03  2574600                 NaN  
       2007-01-04  2073700                 NaN  
       2007-01-05  2676600                 NaN  
       2007-01-08  1557200                 NaN  
       2007-01-09  1386200                 NaN  
...                    ...                 ...  
ZYXI   2020-02-11   196899          32791665.0  
       2020-02-12   261188          32791665.0  
       2020-02-13   120219          32791665.0  
       2020-02-14   199752          32791665.0  
       2020-02-18   290273          32791665.0  

[5982676 rows x 9 columns]

In [171]:
ticker_data.isnull().sum()

SimFinId                    0
Open                     8669
Low                      8908
High                     8477
Close                    8682
Adj. Close               6134
Dividend              5937123
Volume                      0
Shares Outstanding    1226448
dtype: int64

In [172]:
assess_NA(billi_bound)

NameError: name 'billi_bound' is not defined

In [ ]:
ticker_data.info


In [ ]:
billi_bound.stack().head(25)

In [ ]:
billi_bound.head(25)

In [ ]:
billi_bound.describe()

# Balance sheet 

In [175]:
balance_data

Shares (Basic)  Shares (Diluted)  \
Ticker Fiscal Year                                     
A      2008            363000000.0       371000000.0   
       2009            346000000.0       346000000.0   
       2010            347000000.0       353000000.0   
       2011            347000000.0       355000000.0   
       2012            348000000.0       353000000.0   
...                            ...               ...   
ZYXI   2015             31271234.0        31271234.0   
       2016             31271234.0        31271234.0   
       2017             32156000.0        33196000.0   
       2018             32503000.0        34043000.0   
       2019             32439000.0        33963000.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Fiscal Year                                                    
A      2008                                            1.429000e+09   
       2009                                            2.493000e+09   
       2010                                            2.649000e+09   
       2011                                            3.527000e+09   
       2012                                            2.351000e+09   
...                                                             ...   
ZYXI   2015                                            8.000000e+03   
       2016                                            2.470000e+05   
       2017                                            5.565000e+06   
       2018                                            1.012800e+07   
       2019                                            1.404000e+07   

                    Accounts & Notes Receivable   Inventories  \
Ticker Fiscal Year                                              
A      2008                         770000000.0  6.460000e+08   
       2009                         595000000.0  5.520000e+08   
       2010                         869000000.0  7.160000e+08   
       2011                         860000000.0  8.980000e+08   
       2012                         923000000.0  1.014000e+09   
...                                         ...           ...   
ZYXI   2015                           2426000.0  3.050000e+05   
       2016                           3028000.0  1.070000e+05   
       2017                           2185000.0  4.230000e+05   
       2018                           2791000.0  8.370000e+05   
       2019                           5833000.0  2.378000e+06   

                    Total Current Assets  Property, Plant & Equipment, Net  \
Ticker Fiscal Year                                                           
A      2008                 3.182000e+09                      8.240000e+08   
       2009                 3.961000e+09                      8.450000e+08   
       2010                 6.169000e+09                      9.800000e+08   
       2011                 5.569000e+09                      1.006000e+09   
       2012                 4.629000e+09                      1.164000e+09   
...                                  ...                               ...   
ZYXI   2015                 2.766000e+06                      8.010000e+05   
       2016                 3.422000e+06                      5.800000e+05   
       2017                 8.371000e+06                      1.880000e+05   
       2018                 1.432400e+07                      8.190000e+05   
       2019                 2.256600e+07                      8.580000e+05   

                    Long Term Investments & Receivables  \
Ticker Fiscal Year                                        
A      2008                                 206000000.0   
       2009                                 163000000.0   
       2010                                 142000000.0   
       2011                                 117000000.0   
       2012                                 109000000.0   
...                                                 ...   
ZYXI   2015                       

In [176]:

balance_data_cash = balance_data[["Cash, Cash Equivalents & Short Term Investments", 
                                  "Accounts & Notes Receivable", 
                                  'Treasury Stock', 
                                  'Retained Earnings']]


In [177]:
balance_data_cash

Cash, Cash Equivalents & Short Term Investments  \
Ticker Fiscal Year                                                    
A      2008                                            1.429000e+09   
       2009                                            2.493000e+09   
       2010                                            2.649000e+09   
       2011                                            3.527000e+09   
       2012                                            2.351000e+09   
...                                                             ...   
ZYXI   2015                                            8.000000e+03   
       2016                                            2.470000e+05   
       2017                                            5.565000e+06   
       2018                                            1.012800e+07   
       2019                                            1.404000e+07   

                    Accounts & Notes Receivable  Treasury Stock  \
Ticker Fiscal Year                                                
A      2008                         770000000.0   -7.470000e+09   
       2009                         595000000.0   -7.627000e+09   
       2010                         869000000.0   -8.038000e+09   
       2011                         860000000.0   -8.535000e+09   
       2012                         923000000.0   -8.707000e+09   
...                                         ...             ...   
ZYXI   2015                           2426000.0             NaN   
       2016                           3028000.0    0.000000e+00   
       2017                           2185000.0   -2.430000e+05   
       2018                           2791000.0   -3.675000e+06   
       2019                           5833000.0   -3.846000e+06   

                    Retained Earnings  
Ticker Fiscal Year                     
A      2008              2.791000e+09  
       2009              2.760000e+09  
       2010              3.444000e+09  
       2011              4.456000e+09  
       2012              5.505000e+09  
...                               ...  
ZYXI   2015             -9.845000e+06  
       2016             -9.776000e+06  
       2017             -2.411000e+06  
       2018              4.864000e+06  
       2019              1.435600e+07  

[19255 rows x 4 columns]

In [178]:

balane_tan = balance_data[["Property, Plant & Equipment, Net",
                           "Retained Earnings", 
                           "Total Equity"]]


In [179]:
balane_tan

Property, Plant & Equipment, Net  Retained Earnings  \
Ticker Fiscal Year                                                        
A      2008                             8.240000e+08       2.791000e+09   
       2009                             8.450000e+08       2.760000e+09   
       2010                             9.800000e+08       3.444000e+09   
       2011                             1.006000e+09       4.456000e+09   
       2012                             1.164000e+09       5.505000e+09   
...                                              ...                ...   
ZYXI   2015                             8.010000e+05      -9.845000e+06   
       2016                             5.800000e+05      -9.776000e+06   
       2017                             1.880000e+05      -2.411000e+06   
       2018                             8.190000e+05       4.864000e+06   
       2019                             8.580000e+05       1.435600e+07   

                    Total Equity  
Ticker Fiscal Year                
A      2008         2.559000e+09  
       2009         2.514000e+09  
       2010         3.236000e+09  
       2011         4.316000e+09  
       2012         5.185000e+09  
...                          ...  
ZYXI   2015        -4.071000e+06  
       2016        -3.802000e+06  
       2017         4.902000e+06  
       2018         9.291000e+06  
       2019         1.965300e+07  

[19255 rows x 3 columns]

In [180]:
balance_intan = balance_data[
    ['Treasury Stock', 
     'Long Term Investments & Receivables', 
     'Shares (Basic)', 
     'Shares (Diluted)']]

In [181]:
balance_intan

Treasury Stock  Long Term Investments & Receivables  \
Ticker Fiscal Year                                                        
A      2008          -7.470000e+09                          206000000.0   
       2009          -7.627000e+09                          163000000.0   
       2010          -8.038000e+09                          142000000.0   
       2011          -8.535000e+09                          117000000.0   
       2012          -8.707000e+09                          109000000.0   
...                            ...                                  ...   
ZYXI   2015                    NaN                                  NaN   
       2016           0.000000e+00                                  NaN   
       2017          -2.430000e+05                                  NaN   
       2018          -3.675000e+06                                  NaN   
       2019          -3.846000e+06                                  NaN   

                    Shares (Basic)  Shares (Diluted)  
Ticker Fiscal Year                                    
A      2008            363000000.0       371000000.0  
       2009            346000000.0       346000000.0  
       2010            347000000.0       353000000.0  
       2011            347000000.0       355000000.0  
       2012            348000000.0       353000000.0  
...                            ...               ...  
ZYXI   2015             31271234.0        31271234.0  
       2016             31271234.0        31271234.0  
       2017             32156000.0        33196000.0  
       2018             32503000.0        34043000.0  
       2019             32439000.0        33963000.0  

[19255 rows x 4 columns]

In [182]:
balance_data_neg = balance_data[
    ["Inventories",
     "Share Capital & Additional Paid-In Capital", 
     "Payables & Accruals", "Total Noncurrent Liabilities", 
     "Long Term Debt", 
     'Total Liabilities & Equity', 
     'Total Current Liabilities']]

In [183]:
balance_data_neg

Inventories  Share Capital & Additional Paid-In Capital  \
Ticker Fiscal Year                                                             
A      2008         6.460000e+08                                7.416000e+09   
       2009         5.520000e+08                                7.558000e+09   
       2010         7.160000e+08                                7.910000e+09   
       2011         8.980000e+08                                8.271000e+09   
       2012         1.014000e+09                                8.495000e+09   
...                          ...                                         ...   
ZYXI   2015         3.050000e+05                                5.863000e+06   
       2016         1.070000e+05                                6.063000e+06   
       2017         4.230000e+05                                7.645000e+06   
       2018         8.370000e+05                                8.191000e+06   
       2019         2.378000e+06                                9.232000e+06   

                    Payables & Accruals  Total Noncurrent Liabilities  \
Ticker Fiscal Year                                                      
A      2008                 436000000.0                  3.118000e+09   
       2009                 307000000.0                  3.975000e+09   
       2010                 499000000.0                  3.377000e+09   
       2011                 472000000.0                  2.904000e+09   
       2012                 461000000.0                  3.458000e+09   
...                                 ...                           ...   
ZYXI   2015                   3339000.0                  2.280000e+05   
       2016                   3976000.0                  1.480000e+05   
       2017                   2781000.0                  1.200000e+04   
       2018                   5432000.0                  2.977000e+06   
       2019                   3941000.0                  3.427000e+06   

                    Long Term Debt  Total Liabilities & Equity  \
Ticker Fiscal Year                                               
A      2008           2.125000e+09                  7007000000   
       2009           2.904000e+09                  7612000000   
       2010           2.190000e+09                  9696000000   
       2011           1.932000e+09                  9057000000   
       2012           2.112000e+09                 10536000000   
...                            ...                         ...   
ZYXI   2015           2.160000e+05                     3696000   
       2016           1.360000e+05                     4091000   
       2017                    NaN                     8929000   
       2018           2.967000e+06                    19251000   
       2019                    NaN                    28277000   

                    Total Current Liabilities  
Ticker Fiscal Year                             
A      2008                      1.330000e+09  
       2009                      1.123000e+09  
       2010                      3.083000e+09  
       2011                      1.837000e+09  
       2012                      1.893000e+09  
...                                       ...  
ZYXI   2015                      7.539000e+06  
       2016                      7.745000e+06  
       2017                      4.015000e+06  
       2018                      6.983000e+06  
       2019                      5.197000e+06  

[19255 rows x 7 columns]

# income statement

In [184]:

income_neg = income_annual[
    ["Cost of Revenue", 
     "Operating Expenses", 
     "Selling, General & Administrative"]]


In [185]:
income_neg

Cost of Revenue  Operating Expenses  \
Ticker Fiscal Year                                        
A      2008           -2.578000e+09       -2.401000e+09   
       2009           -2.189000e+09       -2.245000e+09   
       2010           -2.514000e+09       -2.364000e+09   
       2011           -3.086000e+09       -2.458000e+09   
       2012           -3.254000e+09       -2.485000e+09   
...                             ...                 ...   
ZYXI   2015           -4.937000e+06       -9.185000e+06   
       2016           -3.517000e+06       -9.156000e+06   
       2017           -4.819000e+06       -9.669000e+06   
       2018           -6.038000e+06       -1.550900e+07   
       2019           -8.814000e+06       -2.559200e+07   

                    Selling, General & Administrative  
Ticker Fiscal Year                                     
A      2008                             -1.697000e+09  
       2009                             -1.603000e+09  
       2010                             -1.752000e+09  
       2011                             -1.809000e+09  
       2012                             -1.817000e+09  
...                                               ...  
ZYXI   2015                             -9.185000e+06  
       2016                             -9.156000e+06  
       2017                             -9.669000e+06  
       2018                             -1.550900e+07  
       2019                             -2.559200e+07  

[19255 rows x 3 columns]

In [186]:
in_income = income_annual[
    ["Net Extraordinary Gains (Losses)", 
     "Research & Development", 
     "Non-Operating Income (Loss)"]]

In [187]:
in_income

Net Extraordinary Gains (Losses)  Research & Development  \
Ticker Fiscal Year                                                             
A      2008                                      NaN            -704000000.0   
       2009                                      NaN            -642000000.0   
       2010                                      NaN            -612000000.0   
       2011                                      NaN            -649000000.0   
       2012                                      NaN            -668000000.0   
...                                              ...                     ...   
ZYXI   2015                                      NaN                     NaN   
       2016                                      NaN                     NaN   
       2017                                      NaN                     NaN   
       2018                                      NaN                     NaN   
       2019                                      NaN                     NaN   

                    Non-Operating Income (Loss)  
Ticker Fiscal Year                               
A      2008                          20000000.0  
       2009                         -40000000.0  
       2010                          -6000000.0  
       2011                         -39000000.0  
       2012                         -76000000.0  
...                                         ...  
ZYXI   2015                           -510000.0  
       2016                           -556000.0  
       2017                          -1450000.0  
       2018                           -154000.0  
       2019                             -5000.0  

[19255 rows x 3 columns]

In [188]:

income_pos = income_annual[
    ["Gross Profit", 
     "Operating Income (Loss)", 
     "Income (Loss) from Continuing Operations", 
     "Net Income", 
     "Net Income (Common)"]]


In [189]:
income_pos

Gross Profit  Operating Income (Loss)  \
Ticker Fiscal Year                                          
A      2008         3.196000e+09                795000000   
       2009         2.292000e+09                 47000000   
       2010         2.930000e+09                566000000   
       2011         3.529000e+09               1071000000   
       2012         3.604000e+09               1119000000   
...                          ...                      ...   
ZYXI   2015         6.704000e+06                 -2481000   
       2016         9.796000e+06                   640000   
       2017         1.861300e+07                  8944000   
       2018         2.587900e+07                 10370000   
       2019         3.665800e+07                 11066000   

                    Income (Loss) from Continuing Operations  Net Income  \
Ticker Fiscal Year                                                         
A      2008                                        693000000   693000000   
       2009                                        -31000000   -31000000   
       2010                                        684000000   684000000   
       2011                                       1012000000  1012000000   
       2012                                       1153000000  1153000000   
...                                                      ...         ...   
ZYXI   2015                                         -2934000    -2911000   
       2016                                            69000       69000   
       2017                                          7365000     7365000   
       2018                                          9552000     9552000   
       2019                                          9492000     9492000   

                    Net Income (Common)  
Ticker Fiscal Year                       
A      2008                   693000000  
       2009                   -31000000  
       2010                   684000000  
       2011                  1012000000  
       2012                  1153000000  
...                                 ...  
ZYXI   2015                    -2911000  
       2016                       69000  
       2017                     7365000  
       2018                     9552000  
       2019                     9492000  

[19255 rows x 5 columns]

# ticker

In [191]:
ticktock_ticker = ticker_data[["Open", "Close", "Adj. Close", "Volume", "Shares Outstanding"]]

In [190]:
ticktock_ticker

Open  Close  Adj. Close   Volume  Shares Outstanding
Ticker Date                                                             
A      2007-01-03  34.99  34.30       22.72  2574600                 NaN
       2007-01-04  34.30  34.41       22.80  2073700                 NaN
       2007-01-05  34.30  34.09       22.58  2676600                 NaN
       2007-01-08  33.98  33.97       22.51  1557200                 NaN
       2007-01-09  34.08  34.01       22.53  1386200                 NaN
...                  ...    ...         ...      ...                 ...
ZYXI   2020-02-11  10.02   9.88        9.88   196899          32791665.0
       2020-02-12   9.92  10.32       10.32   261188          32791665.0
       2020-02-13  10.24  10.26       10.26   120219          32791665.0
       2020-02-14  10.20  10.16       10.16   199752          32791665.0
       2020-02-18  10.15  10.18       10.18   290273          32791665.0

[5982676 rows x 5 columns]

# cashflow

In [116]:

cash_flow_var = cash_flow[
    ["Change in Working Capital",
     "Change in Accounts Receivable",
     "Change in Inventories", 
     "Change in Accounts Payable"]]


In [117]:
cash_flow_var

Change in Working Capital  Change in Accounts Receivable  \
Ticker Fiscal Year                                                             
A      2008                      -179000000.0                    -44000000.0   
       2009                        62000000.0                    193000000.0   
       2010                       -52000000.0                   -166000000.0   
       2011                      -224000000.0                     11000000.0   
       2012                      -176000000.0                     19000000.0   
...                                       ...                            ...   
ZYXI   2015                         1653000.0                            NaN   
       2016                         1073000.0                            NaN   
       2017                        -1162000.0                            NaN   
       2018                         -607000.0                            NaN   
       2019                        -4119000.0                            NaN   

                    Change in Inventories  Change in Accounts Payable  
Ticker Fiscal Year                                                     
A      2008                   -14000000.0                 -21000000.0  
       2009                    47000000.0                  -7000000.0  
       2010                   -51000000.0                 113000000.0  
       2011                  -208000000.0                 -35000000.0  
       2012                   -52000000.0                 -31000000.0  
...                                   ...                         ...  
ZYXI   2015                           NaN                         NaN  
       2016                           NaN                         NaN  
       2017                           NaN                         NaN  
       2018                           NaN                         NaN  
       2019                           NaN                         NaN  

[19238 rows x 4 columns]

In [55]:


cash_flow_negative = cash_flow[
    ["Dividends Paid", 
     "Cash from (Repurchase of) Equity", 
     "Net Cash from Financing Activities"]]



In [55]:
cash_flow_negative

Dividends Paid  Cash from (Repurchase of) Equity  \
Ticker Fiscal Year                                                     
A      2008                    NaN                      -790000000.0   
       2009                    NaN                       -86000000.0   
       2010                    0.0                      -112000000.0   
       2011                    0.0                      -193000000.0   
       2012           -104000000.0                       -72000000.0   
...                            ...                               ...   
ZYXI   2015                    NaN                               NaN   
       2016                    NaN                               0.0   
       2017                    NaN                         -183000.0   
       2018                    NaN                        -3256000.0   
       2019             -2262000.0                           50000.0   

                    Net Cash from Financing Activities  
Ticker Fiscal Year                                      
A      2008                              -7.880000e+08  
       2009                               6.470000e+08  
       2010                               6.010000e+08  
       2011                              -1.693000e+09  
       2012                              -3.500000e+07  
...                                                ...  
ZYXI   2015                              -5.040000e+05  
       2016                              -1.304000e+06  
       2017                              -2.855000e+06  
       2018                              -3.763000e+06  
       2019                              -2.231000e+06  

[19238 rows x 3 columns]

In [114]:


cash_flow_positive = cash_flow[
    ['Net Cash from Operating Activities',
     'Net Cash from Acquisitions & Divestitures', 
     'Net Cash from Investing Activities', 
     'Cash from (Repayment of) Debt']]



In [115]:
cash_flow_positive

Net Cash from Operating Activities  \
Ticker Fiscal Year                                       
A      2008                                  756000000   
       2009                                  408000000   
       2010                                  718000000   
       2011                                 1260000000   
       2012                                 1228000000   
...                                                ...   
ZYXI   2015                                     341000   
       2016                                    1769000   
       2017                                    8260000   
       2018                                    9408000   
       2019                                    6303000   

                    Net Cash from Acquisitions & Divestitures  \
Ticker Fiscal Year                                              
A      2008                                     -1.720000e+08   
       2009                                      4.300000e+07   
       2010                                     -1.108000e+09   
       2011                                     -9.700000e+07   
       2012                                     -2.257000e+09   
...                                                       ...   
ZYXI   2015                                               NaN   
       2016                                               NaN   
       2017                                               NaN   
       2018                                               NaN   
       2019                                               NaN   

                    Net Cash from Investing Activities  \
Ticker Fiscal Year                                       
A      2008                              -3.850000e+08   
       2009                              -4.000000e+06   
       2010                              -1.174000e+09   
       2011                               1.294000e+09   
       2012                              -2.366000e+09   
...                                                ...   
ZYXI   2015                               1.080000e+05   
       2016                              -2.260000e+05   
       2017                              -8.700000e+04   
       2018                              -1.082000e+06   
       2019                              -1.600000e+05   

                    Cash from (Repayment of) Debt  
Ticker Fiscal Year                                 
A      2008                          1.600000e+07  
       2009                          7.480000e+08  
       2010                          7.180000e+08  
       2011                         -1.500000e+09  
       2012                          1.500000e+08  
...                                           ...  
ZYXI   2015                         -5.040000e+05  
       2016                         -1.304000e+06  
       2017                         -2.517000e+06  
       2018                         -5.070000e+05  
       2019                         -1.900000e+04  

[19238 rows x 4 columns]